In [1]:
import sys; sys.path.insert(0, "..")
import re
import operator
from functools import reduce

import pandas as pd
import numpy as np
import deba

deba.set_root('..')

In [4]:
df = pd.read_csv(deba.data('features/minutes.csv'))
df.loc[df.text.str.lower().str.contains('appeal', regex=False)].region.unique()

array(['east_baton_rouge', 'kenner', 'orleans', 'bossier', 'sulphur',
       'louisiana_state', 'slidell', 'carencro', 'mandeville',
       'broussard', 'lake_charles', 'harahan', 'westwego', 'greenwood',
       'westmonroe', 'youngsville', 'monroe'], dtype=object)

In [6]:
df.loc[:, 'text'] = df.text.str.lower()
df.loc[
    df.text.str.contains('appeal', regex=False) &
    ~df.text.str.contains('appeal hearing for', regex=False) &
    (df.region == 'east_baton_rouge')
]

,fileid,pageno,region,pagetype,docpageno,docid,text
131,05aa7d8,1,east_baton_rouge,meeting,NaN,b5f231f8,11:25 504 342 2858 state examiner —. __o02/003...
137,0745275,1,east_baton_rouge,meeting,NaN,977de011,aperove?\n\nminutes\nmunicipal fire and police...
162,0745275,26,east_baton_rouge,meeting,1.0,0aaf2891,minutes\nmunicipal fire and police civil servi...
166,0745275,30,east_baton_rouge,NaN,NaN,0aaf2891,chief phares asked the board to change the req...
209,0fa3166,1,east_baton_rouge,meeting,NaN,940f978e,special meeting\n\nbaton rouge municipal fire ...
...,...,...,...,...,...,...,...
3673,f3ab179,22,east_baton_rouge,NaN,NaN,5875433c,2 positions building service officer\n\n1 posi...
3713,f7c593a,15,east_baton_rouge,notice,2.0,3050f16e,assistant chief training officer (expires 10/2...
3716,f7c593a,18,east_baton_rouge,NaN,5.0,3050f16e,joe lotwick asked if mrs. synder had made an i...
3722,f7c593a,24,east_baton_rouge,NaN,NaN,3050f16e,the board has voted to uphold the disciplinary...


In [3]:
page_cls = df[["fileid", "pageno", "lineno", "text"]].set_index(["fileid", "pageno", "lineno"], drop=False)
page_cls.loc[:, "hd"] = page_cls.lineno <= 3
linenos = df[["fileid", "pageno", "lineno"]].groupby(["fileid", "pageno"]).max().join(page_cls.lineno.to_frame(), how='right', lsuffix='_max')
page_cls.loc[:, "ft"] = linenos.lineno == linenos.lineno_max

page_cls.loc[:, "re_frontpage_1"] = ((page_cls.hd | page_cls.ft) & page_cls.text.str.match(r"page 1$", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_frontpage_2"] = (page_cls.ft & page_cls.text.str.match(r"^1$", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_contpage_1"] = ((page_cls.hd | page_cls.ft) & page_cls.text.str.match(r"page ([2-9]|1[0-9])", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_contpage_2"] = (page_cls.ft & page_cls.text.str.match(r"^([2-9]|1[0-9])$", flags=re.IGNORECASE)).astype(int)

page_cls = page_cls.reset_index(drop=True)
page_cls = page_cls[['fileid', 'pageno']+[col for col in page_cls.columns if col.startswith("re")]].groupby(['fileid', 'pageno']).max()
page_cls
# df[["fileid", "pageno", "lineno"]].groupby(["fileid", "pageno"]).max().reset_index()

KeyError: "['lineno'] not in index"

In [ ]:
hdr_fts = df.loc[df.lineno <= 8, ["region", "fileid", "pageno", "lineno", "text"]].set_index(['fileid','pageno','lineno'])
def match(group:pd.DataFrame):
    region = group.region[0]
    if region not in REGEXES_MAP:
        return group
    for feat, patterns in REGEXES_MAP[region].items():
        group.loc[:, feat] = reduce(operator.or_, [group.text.str.match(pattern) for pattern in patterns], False)
    return group
hdr_fts = hdr_fts.groupby("region").apply(match)
feat_cols = ['mtg', 'hrg', 'agd']
hdr_fts.loc[:, feat_cols] = hdr_fts[feat_cols].fillna(False).astype(int)
hdr_fts = hdr_fts.reset_index(drop=False).groupby(['fileid', 'pageno'])[feat_cols].max()
hdr_fts
    # if region not in REGEXES_MAP:
    #     continue
    # for feat, patterns in REGEXES_MAP[region].items():
    #     for pattern in patterns:
    #         hdr_fts.loc[frame.text.str.match(pattern), feat] = 1
# hdr_fts

mtg  hrg  agd
fileid  pageno               
003b2bf 1         1    0    0
        2         0    0    0
        3         0    0    0
        4         0    0    0
00508a5 1         1    0    0
...             ...  ...  ...
ff43b3c 1         0    0    0
        2         0    0    0
ffe2b9b 1         1    0    0
        2         0    0    0
        3         0    0    0

[3530 rows x 3 columns]

In [ ]:
hdr_fts

mtg  hrg  agd
fileid  pageno               
003b2bf 1         1    0    0
        2         0    0    0
        3         0    0    0
        4         0    0    0
00508a5 1         1    0    0
...             ...  ...  ...
ff43b3c 1         0    0    0
        2         0    0    0
ffe2b9b 1         1    0    0
        2         0    0    0
        3         0    0    0

[3530 rows x 3 columns]

In [ ]:
linenos

lineno_max  lineno
fileid  pageno lineno                    
003b2bf 1      0               34       0
               1               34       1
               2               34       2
               3               34       3
               4               34       4
...                           ...     ...
ffe2b9b 3      3                7       3
               4                7       4
               5                7       5
               6                7       6
               7                7       7

[125855 rows x 2 columns]